## Table of contents

- [Preprocessing](#Preprocessing)
    + [1. 결측치 제거](#Missing-Value-Handling)
    + [2. 중복값 제거](#Remove-duplicates)

- [Analyze](#Analyze)
    + [자치구별 grdp와 학교 및 학원/교습소 수](#grdp와-교육열의-관계)

- [Visualize](#Visualize)
    + [1.자치구별 grdp와 학교 총 개수 시각화](#1.자치구별-grdp와-학교-총-개수-시각화)
    + [2. 자치구별 grdp와 자치구별 학원/교습소 수](#2.자치구별-grdp와-자치구별-학원/교습소-수)


### Preprocessing
전처리 수행(결측값 처리, 중복값 처리, 이상치 처리 등)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

In [6]:
# Main Data
school = pd.read_csv('./data/서울특별시 학교 기본정보.csv')
grdp=pd.read_csv('./data/자치구별 1인당 지역내총생산 및 수준지수.csv')
#전처리 데이터 수합 엑셀 파일 중, '학원교습소정보' 사용
hello= pd.read_excel('./data/helloworld_data_set.xlsx',sheet_name=None)
print(hello.keys())
locals().update(hello)
#학원교습소정보를 academy 변수에 지정
academy= hello['학원교습소정보']

UnicodeDecodeError: 'cp949' codec can't decode byte 0xed in position 0: illegal multibyte sequence

### Missing Value Handling

In [ ]:
school.info()

In [ ]:
school.head()

In [ ]:
school.isnull().sum()

In [ ]:
# 해당 열들은 대부분의 값이 비어있음  -> 유용하지 않은 열이므로 삭제 
# 고등학교구분명, 특수목적고등학교계열명, 계열명, 학과명
del school['고등학교구분명']
del school['특수목적고등학교계열명']
del school['계열명']
del school['학과명']
del school['도로명상세주소']

In [ ]:
school.isnull().sum()

In [ ]:
school['자치구']= school['도로명주소'].str.split().str[1] # 자치구만 뽑아내기
school['자치구']

In [ ]:
school.head()

In [ ]:
# 결측치 행 제거 
school= school.dropna(subset=['도로명주소', '자치구']) 

In [ ]:
school.isnull().sum()

In [ ]:
school['주야과정'].value_counts(dropna=False)

In [ ]:
school['고등학교일반실업구분명'].value_counts(dropna=False)

In [ ]:
school['고등학교일반실업구분명'] = school['고등학교일반실업구분명'].fillna('기타') #nan값을 기타로 채우기
school['주야과정'] = school['주야과정'].fillna('기타') # nan값을 기타로 채우기

In [ ]:
school.isnull().sum()

### 학교 종류 (초중고로 정리)

In [ ]:
# 학교 종류 정리
school['학교종류명'].value_counts()

In [ ]:
#이름 바꾸기
school.loc[school['학교종류명'] == '평생학교(고)-2년6학기','학교종류명'] = '고등학교'
school.loc[school['학교종류명'] == '각종학교(고)','학교종류명'] = '고등학교'
school.loc[school['학교종류명'] == '평생학교(고)-3년6학기','학교종류명'] = '고등학교'
school.loc[school['학교종류명'] == '평생학교(중)-2년6학기','학교종류명'] = '중학교'
school.loc[school['학교종류명'] == '각종학교(중)','학교종류명'] = '중학교'
school.loc[school['학교종류명'] == '방송통신고등학교','학교종류명'] = '고등학교'
school.loc[school['학교종류명'] == '평생학교(초)-3년6학기','학교종류명'] = '초등학교'
school.loc[school['학교종류명'] == '고등기술학교','학교종류명'] = '고등학교'
school.loc[school['학교종류명'] == '평생학교(중)-3년6학기','학교종류명'] = '중학교'
school.loc[school['학교종류명'] == '방송통신중학교','학교종류명'] = '중학교'
school.loc[school['학교종류명'] == '공동실습소','학교종류명'] = '고등학교'
school.loc[school['학교종류명'] == '각종학교(초)','학교종류명'] = '초등학교'

In [ ]:
school['학교종류명'].value_counts(dropna=False)

In [ ]:
school.head()

### Remove duplicates

In [ ]:
school.duplicated().sum()

In [ ]:
school[school.duplicated()]

In [ ]:
school.shape

In [ ]:
school.drop_duplicates(inplace=True) 

In [ ]:
school.shape

In [ ]:
school.info()

In [ ]:
school.head()

In [ ]:
# 분석에 쓰이지 않는 열 삭제
del school['설립구분']
del school['학교명']
del school['관할조직명']
del school['도로명주소']
del school['남녀공학구분명']
del school['산업체특별학급존재여부']
del school['고등학교일반실업구분명']
del school['입시전후기구분명']
del school['주야구분명']
del school['설립일자']
del school['개교기념일']
del school['시도교육청명']
del school['주야과정']

In [ ]:
school.head()

In [ ]:
school.info()

## Analyze

#### gdp와 교육열의 관계

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#자치구별 지역내총생산 groupby
grdp_sum=grdp.groupby('자치구')['지역내총생산(당해년가격)'].sum();grdp_sum 

##### 자치구별 학교수 총계

In [ ]:
#자치구별 학교 합계수
school_count= school['자치구'].value_counts()
school_count=pd.DataFrame(school_count)
school_count=school_count.sort_index();school_count #가나다로 정렬
#열 이름 바꾸기
school_count=school_count.rename(columns={'자치구':'학교수'});school_count

##### 자치구별 학원/교습소 갯수

In [ ]:
# 행정구역명 -> 자치구로 변경
academy_sum=academy.groupby('자치구')[['학원/교습소']].count(); academy_sum #행정구역명별 학원/교습소 groupby

##### 자치구별 grdp/ 학교수/ 학원 교습소 갯수 병합

In [ ]:
#자치구 기준으로 병합
df=pd.concat([grdp_sum,school_count,academy_sum ],axis=1);df
df= df.dropna() #서울시 삭제 (NAN값 삭제)
df

## Visualize

#### 1.자치구별 grdp와 학교 총 개수 시각화

In [ ]:
x = df.index
y1 = df['학교수']
y2 = df['지역내총생산(당해년가격)']

fig = plt.figure(figsize=(8,8)) ## 캔버스 생성
fig.set_facecolor('white')
ax1 = fig.add_subplot() ## axes 생성
 
color1 = 'purple'
ax1.plot(x, y2, color=color1)
ax1.set_xlabel('자치구')
ax1.set_ylabel('지역내총생산(당해년가격)', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)

color2 = 'green'
ax2 = ax1.twinx() # 새로운 y축 생성
ax2.plot(x, y1, color=color2)
ax2.set_ylabel('학교수', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

fig.autofmt_xdate(rotation=45)# x축 45도 회전
plt.title('자치구별 grdp와 자치구별 학교수')

ax1.legend(['grdp총수'],loc='upper left') #범례 달기
ax2.legend(['학교수'],loc='upper right')
plt.show();

# grdp의 움직임에 따라서 자치구별 학교수도 같은 움직임을 나타낸다는 것을 알 수 있었다. 

#### 2. 자치구별 grdp와 자치구별 학원/교습소 수

In [ ]:
x = df.index
y1 = df['학원/교습소']
y2 = df['지역내총생산(당해년가격)']

fig = plt.figure(figsize=(7,7)) ## 캔버스 생성

fig.set_facecolor('white')
ax1 = fig.add_subplot() ## axes 생성
 
color1 = 'green'
ax1.plot(x, y2, color=color1)
ax1.set_xlabel('자치구')
ax1.set_ylabel('grdp총수', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
 
color2 = 'skyblue'
ax2 = ax1.twinx() # 새로운 y축 생성
ax2.plot(x, y1, color=color2)
ax2.set_ylabel('학원/교습소', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)


plt.title('자치구별 grdp와 자치구별 학원/교습소')
fig.autofmt_xdate(rotation=45)
ax1.legend(['grdp총수'],loc='upper left')
ax2.legend(['학원/교습소'],loc='upper right');

## grdp의 변동에 따라서 학원/교습소의 수도 같은 움직임을 보인다.
## 자치구별 gdp가 높은 곳일수록 학교, 학원 및 교습소 수도 많음을 그래프를 통해 확인할 수 있다.